In [1]:
import tensorflow as tf
import numpy as np

In [2]:
X = np.load('/kaggle/input/train-180-127/X_data_third.npy')
Y = np.load('/kaggle/input/train-180-127/Y_data_third.npy')

In [ ]:
type(X[0][0])

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scalar = MinMaxScaler()
X_scaled_train = scalar.fit_transform(X)

In [ ]:
type(X_scaled_train[0][0])

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [5]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [6]:
autoencoder_model = Sequential([
    tf.keras.Input(shape = (22860,)),
    Dense(units = 7620, activation='relu'),
    Dense(units = 2540, activation='relu'),
    Dense(units = 847,  activation='relu'),
    Dense(units = 282,  activation= 'relu'),

    #Dense(units = 282,  activation='relu'),
    Dense(units = 847,  activation='relu'),
    Dense(units = 2540, activation='relu'),
    Dense(units = 7620, activation='relu'),
    Dense(units = 22860,activation='linear')])

autoencoder_model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss = 'MSE')

In [7]:
autoencoder_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 7620)              174200820 
                                                                 
 dense_1 (Dense)             (None, 2540)              19357340  
                                                                 
 dense_2 (Dense)             (None, 847)               2152227   
                                                                 
 dense_3 (Dense)             (None, 282)               239136    
                                                                 
 dense_4 (Dense)             (None, 847)               239701    
                                                                 
 dense_5 (Dense)             (None, 2540)              2153920   
                                                                 
 dense_6 (Dense)             (None, 7620)              1

In [8]:
autoencoder_model.fit(X_train, X_train, epochs =400, batch_size=128, shuffle= True)

Epoch 1/400
142/142 [==============================] - 15s 68ms/step - loss: 174595.9062
Epoch 2/400
142/142 [==============================] - 10s 68ms/step - loss: 1774.8040
Epoch 3/400
142/142 [==============================] - 10s 68ms/step - loss: 1353.5551
Epoch 4/400
142/142 [==============================] - 10s 68ms/step - loss: 1122.9874
Epoch 5/400
142/142 [==============================] - 10s 68ms/step - loss: 1022.5247
Epoch 6/400
142/142 [==============================] - 10s 68ms/step - loss: 949.2602
Epoch 7/400
142/142 [==============================] - 10s 67ms/step - loss: 909.8735
Epoch 8/400
142/142 [==============================] - 10s 68ms/step - loss: 878.3342
Epoch 9/400
142/142 [==============================] - 10s 67ms/step - loss: 908.8916
Epoch 10/400
142/142 [==============================] - 10s 68ms/step - loss: 815.9624
Epoch 11/400
142/142 [==============================] - 10s 68ms/step - loss: 789.4943
Epoch 12/400
142/142 [=======================

In [9]:
encoder_model = Sequential([
    autoencoder_model.layers[0],
    autoencoder_model.layers[1],
    autoencoder_model.layers[2],
    autoencoder_model.layers[3],
])


In [10]:
X_train_encoded = encoder_model.predict(X_train)
X_test_encoded  = encoder_model.predict(X_test)

142/142 [==============================] - 0s 3ms/step


In [11]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(18151, 282)
(4538, 282)


In [12]:
from sklearn.neighbors import KNeighborsClassifier
Knn = KNeighborsClassifier(n_neighbors=51)
Knn.fit(X_train_encoded, y_train)

KNeighborsClassifier(n_neighbors=51)

In [13]:
y_pred = Knn.predict(X_test_encoded)

In [14]:
from sklearn.metrics import confusion_matrix
 
cm = confusion_matrix(y_test, y_pred)

In [15]:
print(cm)

[[447   0   7   2   0   5   1   1   8   0]
 [  4  56   5 132 139   2   4  72   3  83]
 [  0   1 459   1   0   1  10   0   1   0]
 [  1 124   3  55 108   2   3  85   1  87]
 [  4 124   5 102  39   1   4  81   1  79]
 [  7   1   1   0   0 446   4   0   2   0]
 [  4   2  13   0   1   2 436   2   5   0]
 [  3 115   1 116 117   0   0  29   4  65]
 [ 16   0  21   1   0   6  20   0 316   2]
 [  4 126   4 102  99   2   1  66   2  21]]


In [16]:
Knn.score(X_test_encoded,y_test)

0.5077126487439401